In [1]:
#import json
import ast

#Find the SCC around a given pivot state
#by finding an intersection of Forwards and Backwards reachability from pivot

def intersect_FB( visitedF, visitedB ):
    result = {} #will be stored as dictionary {rectangle : {entryF facet : list exitF facets}} in the forward direction
    for rkey in visitedF:
        if rkey in visitedB:
            ffacets = visitedF[ rkey ] # entry-> exits forward
            bfacets = visitedB[ rkey ] # exit -> entries from fw point of view
            rresult = { }# facet:list of exits for this given rectangle
            #compare, store the "intersection"
            r_intersectionF = {} #in the form of entry facet F : list of exit facets F
            for fkey in ffacets:
                #if fkey somewhere in list of some bfkey
                for bfkey in ffacets[ fkey ]: #forward [] of exits
                    if bfkey in bfacets: #is exit listed in bacwards?
                        if fkey in bfacets[ bfkey ]: #does exit backwards lead to fw entry?
                            #store
                            if fkey not in r_intersectionF:
                                r_intersectionF[ fkey ] = []
                            r_intersectionF[ fkey ].append( bfkey )
            result[ rkey ] = r_intersectionF #for rectangle r the intersection of fw and bw flow between facets
    return result

def filter_with_entry_exit_pairs( dictionary ):
    result = {}
    for key in dictionary:
        if dictionary[ key ] != {}:
            result[ key ] = dictionary[ key ]
    return result

def write_out_inter( dictionar, i ):
    #write content to the output file
    fo = open("./outBRUinter/output"+str(i)+".txt", "w")
    #fo.write("Intersection of forward and backward reachable states:\n")
    fo.write( str(dictionar) )
    fo.close()

def read_dict_from_file( fb, i ):
    if fb == "f":
        fo = open("./outBRUforward/output"+str(i)+".txt", "r")
    else:
        fo = open("./outBRUbackward/output"+str(i)+".txt", "r")
    data = fo.read()
    fo.close()
    #js = json.loads(data)
    #with_double_quotes = data.replace( "\'", '"' )
    dictionary = ast.literal_eval( data ) #with_double_quotes )
    return dictionary


In [2]:
#Read forward and backward reachable sets for
#Intersect for rectangles (entry [] the whole rectangle) in init
#write out the results - name them with identifiers "_x_y"
#initial_rectangles = []
identifiers = []
for x in [-3, -2, -1, 0, 1, 2, 3]:
    for y in [-3, -2, -1, 0, 1, 2, 3]:
#        initial_rectangles.append( find_rectangle( [ x, y ] ) )
        identifiers.append( "_" + str( x ) + "_" + str( y ) )

F_visited_sets = {}
F_union_visited = {}
B_visited_sets = {}
B_union_visited = {}

#for r,i in zip( initial_rectangles, identifiers ):
for i in identifiers:
    visitedF = read_dict_from_file( "f", "Konec"+i )
    F_visited_sets[ i ] = visitedF
    F_union_visited |= visitedF
    
    visitedB = read_dict_from_file( "b", "Konec"+i )
    B_visited_sets[ i ] = visitedB
    B_union_visited |= visitedB
        
    intersect = intersect_FB( visitedF, visitedB )
    write_out_inter( intersect, i )
    intersect = filter_with_entry_exit_pairs( intersect )
    print( intersect )
    #add initial rectangles?

write_out_inter( F_union_visited, "UnionForward" )
write_out_inter( B_union_visited, "UnionBackward" )


{}


In [ ]:
#visualize the sets, write png images
#need the tresholds and rectangle->geom real
#import
import numpy as np
import seaborn as sns
# Matplotlib for additional customization
from matplotlib import pyplot as plt
%matplotlib inline

import pandas as pd

def r_hash( h ):
    r = [0,0] #default value
    hparts = h.split('|')
    r[0] = int( hparts[0] )
    r[1] = int( hparts[1] )
    return r

#model-specific tresholds
x0min = -8
x0max = 8
x1min = -8
x1max = 8

pieces_count = 100

stepx0 = ( x0max - x0min ) / pieces_count
stepx1 = ( x1max - x1min ) / pieces_count

tresholdsX0 = np.arange ( x0min, x0max+(0.3*stepx0), stepx0 )
tresholdsX1 = np.arange ( x1min, x1max+(0.3*stepx1), stepx1 )

tresholds = [ tresholdsX0, tresholdsX1 ]

def visualize_rectangles( rdict, i ):
    list_r = []
    
    for key in rdict:
        r = r_hash( key )
        list_r.append( r )
    
    xlist = []
    ylist = []
    for r in list_r:
        xlist.append( 0.5*( tresholds[0][r[0]] + tresholds[0][r[0]+1]) )
        ylist.append( 0.5*( tresholds[1][r[1]] + tresholds[1][r[1]+1]) )

    ctverecky = pd.DataFrame({
        "x" : xlist,
        "y" : ylist
    })

    wid = 5
    hei = 5
    fig,ax = plt.subplots(1, 1, figsize=(wid, hei))
    sns.scatterplot(x="x", y="y", data=ctverecky, markers="s", s=10)
    ax.set(xlim=(x0min, x0max))
    ax.set(ylim=(x1min, x1max))
    plt.savefig('./outVDPinter/img'+str(i)+'.png', dpi=300)
    plt.clf()
    plt.close()

for i in identifiers:
    visitedF = F_visited_sets[ i ]
    visualize_rectangles( visitedF, "Forw"+i )
    visitedB = B_visited_sets[ i ]
    visualize_rectangles( visitedF, "Backw"+i )
        
    intersect = intersect_FB( visitedF, visitedB )
    write_out_inter( intersect, i )
    intersect = filter_with_entry_exit_pairs( intersect )
    visualize_rectangles( intersect, "Intersection"+i )
    #add initial rectangles?
    
visualize_rectangles( F_union_visited, "UnionForward" )
visualize_rectangles( B_union_visited, "UnionBackward" )
